In [1]:
import argparse, os
import numpy as np, ephem
from numpy.lib import recfunctions
from pixell import utils, enmap, bunch, reproject, colors, coordinates, mpi
from scipy import interpolate, optimize
import glob
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use('pdf')

from pathlib import Path
from asteroid_utils_pixell import minorplanet, get_desig

[nia-jupyter.scinet.local:51032] pml_ucx.c:285  Error: UCP worker does not support MPI_THREAD_MULTIPLE


In [2]:
def in_box(box, point): #checks if points are inside box or not
        box   = np.asarray(box)
        point = np.asarray(point)
        point[1] = utils.rewind(point[1], box[0,1])
        # This assumes reverse RA axis in box
        return point[0] > box[0,0] and point[0] < box[1,0] and point[1] > box[1,1] and point[1] < box[0,1]

def make_box(point, rad): #making box
        box     = np.array([point - rad, point + rad])
        box[:,1]= box[::-1,1] # reverse ra
        return box

def filter_map(map, lknee=3000, alpha=-3, beam=0): #filtering map somehow (FFT)
        fmap  = enmap.fft(map)
        l     = np.maximum(0.5, map.modlmap())
        filter= (1+(l/lknee)**alpha)**-1
        if beam:
                filter *= np.exp(-0.5*l**2*beam**2)
        fmap *= filter
        omap  = enmap.ifft(fmap).real
        return omap

def calc_obs_ctime(orbit, tmap, ctime0):
        def calc_chisq(x):
                ctime = ctime0+x
                try:
                        adata = orbit(ctime)
                        mtime = tmap.at(adata[1::-1], order=1)
                except ValueError:
                        mtime = 0
                return (mtime-ctime)**2
        ctime = optimize.fmin_powell(calc_chisq, 0, disp=False)+ctime0
        err   = calc_chisq(ctime-ctime0)**0.5
        return ctime, err

def calc_sidereal_time(lon, ctime):
        obs      = ephem.Observer()
        obs.lon  = lon
        obs.date = utils.ctime2djd(ctime)
        return obs.sidereal_time()
    
def geocentric_to_site(pos, dist, site_pos, site_alt, ctime):
        """Given a geocentric position pos[{ra,dec},...] and distance dist [...]
        in m, transform it to coordinates relative to the given site, with
        position pos[{lon,lat}] and altitude alt_site in m, returns the
        position observed from the site, as well as the distance from the site in m"""
        # This function isn't properly debugged. I should check the sign of
        # the sidereal time shift. But anyway, this function is a 0.2 arcmin
        # effect in the asteroid belt.
        sidtime    = calc_sidereal_time(site_pos[0], ctime)
        site_radec = np.array([site_pos[0]+sidtime*15*utils.degree,site_pos[1]])
        vec_site   = utils.ang2rect(site_radec)*(utils.R_earth + site_alt)
        vec_obj    = utils.ang2rect(pos)*dist
        vec_rel  = vec_obj-vec_site
        dist_rel = np.sum(vec_rel**2,0)**0.5
        pos_rel  = utils.rect2ang(vec_rel)
        return pos_rel, dist_rel

In [3]:
comm    = mpi.COMM_WORLD
verbose = 3

#Radius of output thumbnails
r_thumb = 10.0*utils.arcmin
# ... and temporary radius to use before we know the true position
r_full  = r_thumb + 30*utils.arcmin

In [4]:
desig, name, semimajor = get_desig(0)

In [5]:
lknee    = 1500
alpha    = -3.5
beam     = 0*utils.fwhm*utils.arcmin
site     = coordinates.default_site
site_pos = np.array([site.lon,site.lat])*utils.degree
time_tol  = 60
time_sane = 3600*24

In [17]:
# Expand any globs in the input file names

ifiles = ['/home/r/rbond/sigurdkn/project/actpol/maps/depth1/release/*/*_map.fits']
ifiles  = sum([sorted(utils.glob(ifile)) for ifile in ifiles],[])

In [18]:
info    = np.load('/gpfs/fs0/project/r/rbond/sigurdkn/actpol/ephemerides/objects/{}.npy'.format(name.capitalize())).view(np.recarray)

In [19]:
odir = "/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids"

orbit   = interpolate.interp1d(info.ctime, [utils.unwind(info.ra*utils.degree), info.dec*utils.degree, info.r, info.ang*utils.arcsec], kind=3)
utils.mkdir(odir)

print('Starting {}'.format(name))


Starting Vesta


In [21]:
for fi in range(comm.rank, len(ifiles), comm.size):
        ifile    = ifiles[fi]
        infofile = utils.replace(ifile, "map.fits", "info.hdf")
        tfile    = utils.replace(ifile, "map.fits", "time.fits")
        varfile    = utils.replace(ifile, "map.fits", "ivar.fits")
        rhofile    = utils.replace(ifile, "map.fits", "rho.fits")
        kfile    = utils.replace(ifile, "map.fits", "kappa.fits")

        ofname   = "%s/%s/%s_%s" % (odir, name.capitalize(), name, os.path.basename(ifile))
        varname   = "%s/%s/%s_%s" % (odir, name.capitalize(), name, os.path.basename(varfile))
        rhoname   = "%s/%s/%s_%s" % (odir, name.capitalize(), name, os.path.basename(rhofile))
        kname   = "%s/%s/%s_%s" % (odir, name.capitalize(), name, os.path.basename(kfile))
        infoname = "%s/%s/%s_%s" % (odir, name.capitalize(), name, os.path.basename(infofile))

        info     = bunch.read(infofile)
        # Get the asteroid coordinates
        ctime0   = np.mean(info.period)
        adata0   = orbit(ctime0)
        ast_pos0 = utils.rewind(adata0[1::-1])
        message  = "%.0f  %8.3f %8.3f  %8.3f %8.3f %8.3f %8.3f" % (info.t, ast_pos0[1]/utils.degree, ast_pos0[0]/utils.degree, info.box[0,1]/utils.degree, info.box[1,1]/utils.degree, info.box[0,0]/utils.degree, info.box[1,0]/utils.degree)
        # Check if we're in bounds
        if not in_box(info.box, ast_pos0):
                if verbose >= 3:
                        print(colors.lgray + message + " outside" + colors.reset)
                        continue
        #Ok, should try to read in this map. Decide on
        # bounding box to read in


        full_box  = make_box(ast_pos0, r_full)
        # Read it and check if we have enough hits in the area we want to use
        try:
                tmap = enmap.read_map(tfile, box=full_box)
                tmap[tmap!=0] += info.t
        except (TypeError, FileNotFoundError):
                print("Error reading %s. Skipping" % ifile)
                continue
        # Break out early if nothing is hit
        if np.all(tmap == 0): continue
                #if verbose >= 2:
                #       print(colors.white + message + " unhit" + colors.reset)
                #continue
        # Figure out what time the asteroid was actually observed
        ctime, err = calc_obs_ctime(orbit, tmap, ctime0)
        time, err = calc_obs_ctime(orbit, tmap, ctime0)
        if err > time_tol or abs(ctime-ctime0) > time_sane:
                if verbose >= 2:
                        print(colors.white + message + " time" + colors.reset)
                continue
        # Now that we have the proper time, get the asteroids actual position
        adata    = orbit(ctime)
        ast_pos  = utils.rewind(adata[1::-1]).reshape((2,))
        # optionally transform to topocentric here. ~0.1 arcmin effect
        thumb_box = make_box(ast_pos, r_thumb)

        # Read the actual data
        try:
                imap = enmap.read_map(ifile, box=full_box)
                var = enmap.read_map(varfile, box=full_box)
                rho = enmap.read_map(rhofile, box=full_box)
                kap = enmap.read_map(kfile, box=full_box)
        except Exception as e:
                print(colors.red + str(e) + colors.reset)
                continue

        if np.mean(imap.submap(thumb_box) == 0) > 0.0:
                if verbose >= 2:
                        print(colors.white + message + " unhit" + colors.reset)
                        continue
        # Filter the map
        wmap     = filter_map(imap, lknee=lknee, alpha=alpha, beam=beam)
        # And reproject it
        omap = reproject.thumbnails(wmap, ast_pos, r=r_thumb)
        enmap.write_map(ofname, omap)

        vmap = reproject.thumbnails(var, ast_pos, r=r_thumb)
        enmap.write_map(varname, vmap)

        rmap = reproject.thumbnails(rho, ast_pos, r=r_thumb)
        enmap.write_map(rhoname, rmap)

        kmap = reproject.thumbnails(kap, ast_pos, r=r_thumb)
        enmap.write_map(kname, kmap)
        
        try:
            print(ctime[0])
        except:
            continue
        print(kname)
        info.ctime_ast = ctime[0]
        info.ctime_err = err
        bunch.write(infoname, info)
        
        if verbose >= 1:
                print(colors.lgreen + message + " ok" + colors.reset)
print('Finished')

1494463440   125.749   24.100   187.032  127.215   -6.172   22.224 outside
1494463440   125.749   24.100   187.032  127.215   -6.172   22.224 outside
1494463440   125.749   24.100   188.358  127.793   -5.041   22.578 outside
1494463440   125.749   24.100   188.358  127.793   -5.041   22.578 outside
1494463440   125.749   24.100   188.093  128.528   -6.225   21.288 outside
1494463440   125.749   24.100   188.093  128.528   -6.225   21.288 outside
1494478923   125.801   24.089  -132.007 -168.114   -6.140   22.200 outside
1494478923   125.801   24.089  -130.680 -167.533   -5.011   22.551 outside
1494478923   125.801   24.089  -130.680 -167.533   -5.011   22.551 outside
1494478923   125.801   24.089  -130.945 -166.800   -6.196   21.261 outside
1494478923   125.801   24.089  -130.945 -166.800   -6.196   21.261 outside
1494479992   125.830   24.082   -15.373  -80.090  -60.183  -17.134 outside
1494479992   125.830   24.082   -15.373  -80.090  -60.183  -17.134 outside
1494479992   125.830   24

1495170236   128.742   23.426   -77.323 -164.714   -4.870   22.420 outside
1495170236   128.742   23.426   -77.323 -164.714   -4.870   22.420 outside
1495170236   128.742   23.426   -77.180 -163.951   -6.034   21.101 outside
1495170236   128.742   23.426   -77.180 -163.951   -6.034   21.101 outside
1495187454   128.794   23.414    32.601    7.190  -60.013  -16.964 outside
1495187454   128.794   23.414    32.601    7.190  -60.013  -16.964 outside
1495187454   128.794   23.414    32.618    6.207  -61.195  -18.188 outside
1495187454   128.794   23.414    32.618    6.207  -61.195  -18.188 outside
1495187454   128.794   23.414    31.126    4.472  -60.167  -17.832 outside
1495187454   128.794   23.414    31.126    4.472  -60.167  -17.832 outside
1495191992   128.817   23.409    58.576   30.874  -50.706  -28.322 outside
1495191992   128.817   23.409    58.576   30.874  -50.706  -28.322 outside
1495191992   128.817   23.409    58.594   30.495  -51.965  -29.582 outside
1495191992   128.817   23

1495340007   129.472   23.253   -28.648  -93.781  -60.551  -17.104 outside
1495340007   129.472   23.253   -28.648  -93.781  -60.551  -17.104 outside
1495340007   129.472   23.253   -28.633  -95.564  -61.472  -18.335 outside
1495340007   129.472   23.253   -28.633  -95.564  -61.472  -18.335 outside
1495340007   129.472   23.253   -30.024  -96.380  -60.209  -17.953 outside
1495340007   129.472   23.253   -30.024  -96.380  -60.209  -17.953 outside
1495351876   129.507   23.245   -63.072 -118.016   -6.189   21.173 outside
1495351876   129.507   23.245   -63.072 -118.016   -6.189   21.173 outside
1495351876   129.507   23.245   -61.709 -117.409   -5.075   21.252 outside
1495351876   129.507   23.245   -61.709 -117.409   -5.075   21.252 outside
1495351876   129.507   23.245   -62.268 -116.701   -6.276   20.057 outside
1495351876   129.507   23.245   -62.268 -116.701   -6.276   20.057 outside
1495356466   129.524   23.241     4.210  -11.064   -3.379    6.323 outside
1495356466   129.524   23

1500056465   152.160   16.437   191.082  167.917    2.968   20.386 outside
1500056465   152.160   16.437   191.082  167.917    2.968   20.386 outside
1500058246   152.203   16.421   248.295  162.460    3.778   21.149 outside
1500058246   152.203   16.421   248.295  162.460    3.778   21.149 outside
1500058246   152.203   16.421   249.095  163.718    2.775   20.697 outside
1500058246   152.203   16.421   249.095  163.718    2.775   20.697 outside
1500077130   152.310   16.383   281.503  176.108   -6.011   21.762 outside
1500077130   152.310   16.383   281.503  176.108   -6.011   21.762 outside
1500077130   152.310   16.383   282.669  176.657   -4.868   22.440 outside
1500077130   152.310   16.383   282.669  176.657   -4.868   22.440 outside
1500077130   152.310   16.383   282.813  177.420   -6.032   21.122 outside
1500077130   152.310   16.383   282.813  177.420   -6.032   21.122 outside
1500097639   152.371   16.361    17.082  -22.547  -20.384   21.750 outside
1500097639   152.371   16

1500249993   153.184   16.068   -63.080 -173.304   -6.191   21.252 outside
1500249993   153.184   16.068   -63.080 -173.304   -6.191   21.252 outside
1500249993   153.184   16.068   -61.720 -172.699   -5.077   21.331 outside
1500249993   153.184   16.068   -61.720 -172.699   -5.077   21.331 outside
1500249993   153.184   16.068   -62.278 -171.990   -6.278   20.137 outside
1500249993   153.184   16.068   -62.278 -171.990   -6.278   20.137 outside
1500267640   153.263   16.040    -8.321 -105.567  -22.499   21.116 outside
1500267640   153.263   16.040    -8.321 -105.567  -22.499   21.116 outside
1500267640   153.263   16.040    -6.957 -105.301  -21.252   21.205 outside
1500267640   153.263   16.040    -6.957 -105.301  -21.252   21.205 outside
1500267640   153.263   16.040    -7.504 -104.216  -22.176   20.006 outside
1500267640   153.263   16.040    -7.504 -104.216  -22.176   20.006 outside
1500283181   153.328   16.017    80.187    8.182  -21.570   21.181 outside
1500283181   153.328   16

...

1603757365   154.568   13.632   -38.307  -79.096  -22.162   20.510 outside
1603758670   154.620   13.618   120.267   12.433  -61.610  -16.886 outside
1603758670   154.620   13.618   120.267   12.433  -61.610  -16.886 outside
1603758670   154.620   13.618   120.286   10.768  -62.611  -18.114 outside
1603758670   154.620   13.618   120.286   10.768  -62.611  -18.114 outside
1603794816.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1603781218_pa4_f150_kappa.fits
1603781218   154.704   13.595   179.673   80.774   -4.240   22.337 ok
1603794816.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1603781218_pa4_f220_kappa.fits
1603781218   154.704   13.595   179.673   80.774   -4.240   22.337 ok
1603794816.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1603781218_pa5_f090_kappa.fits
1603781218   154.704   13.595   180.304   82.085   -5.359   21.996 ok
1603794816.0
/scratch/r/rbond/jorlo/actxminorplanets/si

1603974382   155.528   13.373   206.246  112.044   -6.270   21.620 unhit
1603980928.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1603974382_pa4_f220_kappa.fits
1603974382   155.528   13.373   206.246  112.044   -6.270   21.620 ok
1603980928.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1603974382_pa5_f090_kappa.fits
1603974382   155.528   13.373   207.548  112.616   -5.133   22.038 ok
1603980928.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1603974382_pa5_f150_kappa.fits
1603974382   155.528   13.373   207.548  112.616   -5.133   22.038 ok
1603974382   155.528   13.373   207.414  113.474   -6.246   20.667 unhit
1603974382   155.528   13.373   207.414  113.474   -6.246   20.667 unhit
1603989842   155.573   13.361   -70.832 -127.491   -4.337   22.279 outside
1603989842   155.573   13.361   -70.832 -127.491   -4.337   22.279 outside
1603989842   155.573   13.361   -70.219 -126.197   -5.449   2

1604127604   156.179   13.198   183.791   82.214   -4.291   20.931 unhit
1604140672.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1604127604_pa4_f220_kappa.fits
1604127604   156.179   13.198   183.791   82.214   -4.291   20.931 ok
1604140672.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1604127604_pa5_f090_kappa.fits
1604127604   156.179   13.198   184.442   83.549   -5.396   20.745 ok
1604140672.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1604127604_pa5_f150_kappa.fits
1604127604   156.179   13.198   184.442   83.549   -5.396   20.745 ok
1604127604   156.179   13.198   183.065   82.721   -5.570   19.590 unhit
1604127604   156.179   13.198   183.065   82.721   -5.570   19.590 unhit
1604155264.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1604143778_pa4_f150_kappa.fits
1604143778   156.250   13.179   194.128   94.261   -6.172   21.853 ok
1604155392.0
/scratch

1604318145   156.976   12.984   211.001  111.141   -6.298   20.484 time
1604324992.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1604318145_pa4_f220_kappa.fits
1604318145   156.976   12.984   211.001  111.141   -6.298   20.484 ok
1604324992.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1604318145_pa5_f090_kappa.fits
1604318145   156.976   12.984   212.337  111.735   -5.175   20.751 ok
1604324992.0
/scratch/r/rbond/jorlo/actxminorplanets/sigurd/asteroids/Vesta/Vesta_depth1_1604318145_pa5_f150_kappa.fits
1604318145   156.976   12.984   212.337  111.735   -5.175   20.751 ok
1604318145   156.976   12.984   212.027  112.571   -6.304   19.409 unhit
1604318145   156.976   12.984   212.027  112.571   -6.304   19.409 unhit
1604334279   157.017   12.973   -77.202 -116.180   -4.145   22.964 outside
1604334279   157.017   12.973   -77.202 -116.180   -4.145   22.964 outside
1604334279   157.017   12.973   -76.614 -114.996   -5.281   22

KeyboardInterrupt: 